In [20]:
import joblib
import pandas
import requests
from PIL import Image
import io, time

 Read Image,
 Get OCR, 
 Get Features,
 Inference,
 Output

In [3]:
keys = ["INCOME","TAX","DEPARTMENT","GOVT","INDIA","PERMANENT","ACCOUNT","NUMBER","SIGNATURE","GOVERNMENT","DOB",	"MALE",	"YEAR",	"BIRTH","UNIQUE","IDENTIFICATION","AUTHORITY","AADHAAR","EXPIRY","ISSUE","SEX","NATIONALITY","GIVEN","SURNAME","<<<", "PASSPORT","TYPE","COUNTRY","CODE"]

In [4]:
def ocr(img,mime_type='image/jpeg'):
    url = "https://centralindia.api.cognitive.microsoft.com/vision/v3.2/read/analyze"
    subscription_key = 'd85c088ad95a42ff8641d0440486311f'
    payload={}
    # files=[('image',('tmp.jpg',image_to_byte_array(img),mime_type))]
    files=[('image',('tmp.jpg',img,mime_type))]
    headers = {'Ocp-Apim-Subscription-Key': subscription_key}
    response = requests.request("POST", url, headers=headers, data=payload, files=files, verify=False)
    status_code = response.status_code
    ocr_out=response.headers
    if status_code== 202:
        url=ocr_out['Operation-Location']
        response = requests.request("GET", url, headers=headers, data=payload, verify=False)
        ocr_out=response.json()
        while ocr_out["status"]=="running":
            time.sleep(1)
            response = requests.request("GET", url, headers=headers, data=payload, verify=False)
            ocr_out=response.json()
    return dict(ocr_out)

In [13]:
def read_img(filename, directory):
    pil_im = Image.open(directory + "//" + filename)
    f = open("test/" + filename + ".json","w")
    ocr_output = str(ocr(image_to_byte_array(pil_im)))
    f.write(ocr_output)
    f.close()
    ls = get_dataset(ocr_output)
    return ls

In [16]:
def image_to_byte_array(image: Image) -> bytes:
  # BytesIO is a file-like buffer stored in memory
  imgByteArr = io.BytesIO()
#   print("Image is being saved.")
  # image.save expects a file-like as a argument
  image.save(imgByteArr, format=image.format)
#   print("Image got saved.")
  # Turn the BytesIO object back into a bytes object
  imgByteArr = imgByteArr.getvalue()
#   print("Image converted to array.")
  return imgByteArr

In [24]:
def get_dataset(ocr_output):
    ls = []
    for k in keys:
        if k in ocr_output:
            ls.append(1)
        else:
            ls.append(0)
    return ls

In [38]:
t = time.time()
model = joblib.load('model.pkl')
ls = read_img("driving-licence-delhi-2.jpeg","test")
predict = model.predict([ls])[0]
if sum(ls) < 4:
    print("Unable to Identify the document category.")
elif predict == 0:
    print("This is PASSPORT.")
elif predict == 1:
    print("This is AADHAAR.")
else:
    print("This is PAN CARD.")
print("Job completed in " + str(int(time.time() - t)) + " sec.")

/Users/30088029/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'centralindia.api.cognitive.microsoft.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/30088029/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'centralindia.api.cognitive.microsoft.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/30088029/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'centralindia.api.cognitive.microsoft.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.i

Unable to Identify the document category.
Job completed in 2 sec.


/Users/30088029/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


5